In [9]:
library("openxlsx")
library("ggplot2")
library("tidyr")
library("dplyr")

In [25]:
# Read the data
data <- read.xlsx("econ.xlsx")

#variáveis X1= gcp e X2= pop para anos igual ou superiores a 1977
X1 <- data[115:574,2]
X2 <- data[115:574,3]

In [27]:
X2

[1] 219179.0 219344.0 219504.0 219684.0 219859.0 220046.0 220239.0 220458.0
  [9] 220688.0 220904.0 221109.0 221303.0 221477.0 221629.0 221792.0 221991.0
 [17] 222176.0 222379.0 222585.0 222805.0 223053.0 223271.0 223477.0 223670.0
 [25] 223865.0 224053.0 224235.0 224438.0 224632.0 224843.0 225055.0 225295.0
 [33] 225547.0 225801.0 226027.0 226243.0 226451.0 226656.0 226849.0 227061.0
 [41] 227251.0 227522.0 227726.0 227953.0 228186.0 228417.0 228612.0 228779.0
 [49] 228937.0 229071.0 229224.0 229403.0 229575.0 229761.0 229966.0 230187.0
 [57] 230412.0 230641.0 230822.0 230989.0 231157.0 231313.0 231470.0 231645.0
 [65] 231809.0 231992.0 232188.0 232392.0 232599.0 232816.0 232993.0 233160.0
 [73] 233322.0 233473.0 233613.0 233781.0 233922.0 234118.0 234307.0 234501.0
 [81] 234701.0 234907.0 235078.0 235235.0 235385.0 235527.0 235675.0 235839.0
 [89] 235993.0 236160.0 236348.0 236549.0 236760.0 236976.0 237159.0 237316.0
 [97] 237468.0 237602.0 237732.0 237900.0 238074.0 238270.0 238466.0 238679.0
[105] 238898.0 239113.0 239307.0 239477.0 239638.0 239788.0 239928.0 240094.0
[113] 240271.0 240459.0 240651.0 240854.0 241068.0 241274.0 241467.0 241620.0
[121] 241784.0 241930.0 242079.0 242252.0 242423.0 242608.0 242804.0 243012.0
[129] 243223.0 243446.0 243639.0 243809.0 243981.0 244131.0 244279.0 244445.0
[137] 244610.0 244806.0 245021.0 245240.0 245464.0 245693.0 245884.0 246056.0
[145] 246224.0 246378.0 246530.0 246721.0 246906.0 247114.0 247342.0 247573.0
[153] 247816.0 248067.0 248281.0 248479.0 248659.0 248827.0 249012.0 249306.0
[161] 249565.0 249849.0 250132.0 250439.0 250751.0 251057.0 251346.0 251626.0
[169] 251889.0 252135.0 252372.0 252643.0 252913.0 253207.0 253493.0 253807.0
[177] 254126.0 254435.0 254718.0 254964.0 255214.0 255448.0 255703.0 255992.0
[185] 256285.0 256589.0 256894.0 257232.0 257548.0 257861.0 258147.0 258413.0
[193] 258679.0 258919.0 259152.0 259414.0 259680.0 259963.0 260255.0 260566.0
[201] 260867.0 261163.0 261425.0 261674.0 261919.0 262123.0 262352.0 262631.0
[209] 262877.0 263152.0 263436.0 263724.0 264017.0 264301.0 264559.0 264804.0
[217] 265044.0 265270.0 265495.0 265755.0 265998.0 266270.0 266557.0 266843.0
[225] 267152.0 267456.0 267715.0 267943.0 268151.0 268364.0 268595.0 268853.0
[233] 269108.0 269386.0 269667.0 269976.0 270284.0 270581.0 270878.0 271125.0
[241] 271360.0 271585.0 271821.0 272083.0 272342.0 272622.0 272912.0 273237.0
[249] 273553.0 273852.0 274126.0 274372.0 274626.0 274838.0 275047.0 275304.0
[257] 275564.0 275836.0 276115.0 276418.0 276714.0 277003.0 277277.0 277526.0
[265] 277790.0 277992.0 278198.0 278451.0 278717.0 279001.0 279295.0 279602.0
[273] 279903.0 280203.0 280471.0 280716.0 280976.0 281190.0 281409.0 281653.0
[281] 281877.0 282126.0 282385.0 282653.0 282932.0 283201.0 283453.0 283696.0
[289] 283920.0 284137.0 284350.0 284581.0 284810.0 285062.0 285309.0 285570.0
[297] 285843.0 286098.0 286341.0 286570.0 286788.0 286994.0 287190.0 287397.0
[305] 287623.0 287864.0 288105.0 288360.0 288618.0 288870.0 289106.0 289313.0
[313] 289518.0 289714.0 289911.0 290125.0 290346.0 290584.0 290820.0 291072.0
[321] 291321.0 291574.0 291807.0 292008.0 292192.0 292368.0 292561.0 292779.0
[329] 292997.0 293223.0 293463.0 293719.0 293971.0 294230.0 294466.0 294694.0
[337] 294914.0 295105.0 295287.0 295490.0 295704.0 295936.0 296186.0 296440.0
[345] 296707.0 296972.0 297207.0 297431.0 297647.0 297854.0 298060.0 298281.0
[353] 298496.0 298739.0 298996.0 299263.0 299554.0 299835.0 300094.0 300340.0
[361] 300574.0 300802.0 301021.0 301254.0 301483.0 301739.0 302004.0 302267.0
[369] 302546.0 302807.0 303054.0 303287.0 303506.0 303711.0 303907.0 304117.0
[377] 304323.0 304556.0 304798.0 305045.0 305309.0 305554.0 305786.0 306004.0
[385] 306208.0 306402.0 306588.0 306787.0 306984.0 307206.0 307439.0 307685.0
[393] 307946.0 308189.0 308418.0 308633.0 308833.0 309027.0 309212.0 309191.2
[401] 309369.1 309548.5 309745.7 309957.8 310176.5 310400.0 310595.8 310781.7
[409] 310960.7 311113.4 